# Fake News Classifier : Deep Learning

Libraries: 

    Link: https://www.kaggle.com/c/fake-news/data
    Dataset: 
        id: unique id for a news article
        title: the title of a news article
        author: author of the news article
        text: the text of the article; could be incomplete
        label: a label that marks the article as potentially unreliable
            1: unreliable
            0: reliable

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('fake-news/train.csv')

In [19]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
print(df.shape)

(20800, 5)


In [21]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [22]:
# Drop Nan Values
df = df.dropna()

In [23]:
X = df.drop('label',axis=1)
Y = df['label']

In [24]:
print(X.shape, Y.shape)

(18285, 4) (18285,)


In [25]:
import tensorflow as tf

In [26]:
tf.__version__

'2.7.0'

In [27]:
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [28]:
# Vocabulary Size
voc_size = 5000

In [29]:
messages = X.copy()

In [30]:
messages['title'][1:3]

1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
Name: title, dtype: object

In [31]:
# we have Dropped Nan values before so, reset_index() method sets a list of 
# integer ranging from 0 to length of data as index
messages.reset_index(inplace=True)

### Data Preprocessing 

In [32]:
import nltk 
import re
from nltk.corpus import stopwords

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sajwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    # print(i)
    review = re.sub('[^a-zA-z]',' ', messages['title'][i])
    # substituting everything with blank except alphabets
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
# First for each index sentence, except all the alphabets all the words will be subsituted to ' '
# then conver each word into lowercase and split each word separately in the sentence.
# for each word not present in stopwords then only that word is stored in review
# again join the words that were splitted for forming a sentence and append together each sentence

In [35]:
corpus[:5]
# all the non necessary word is removed

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

### One Hot Representation

In [36]:
onehotRep = [one_hot(words, voc_size) for words in corpus]
print(onehotRep[:5])

[[2163, 2423, 4250, 687, 4939, 4620, 3891, 2358, 3467, 298], [4890, 466, 1320, 914, 2324, 2649, 821], [4551, 2588, 143, 2559], [2788, 778, 3844, 4269, 1582, 2154], [2787, 2324, 3796, 1020, 425, 32, 2324, 3889, 3669, 3939]]


### Embedding Representation

In [37]:
# All the sent have variable len; using pad_sequence fix the len
sent_len = 20
embedded_docs = pad_sequences(onehotRep, padding = 'pre', maxlen = sent_len)
print(embedded_docs[:5])

[[   0    0    0    0    0    0    0    0    0    0 2163 2423 4250  687
  4939 4620 3891 2358 3467  298]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 4890
   466 1320  914 2324 2649  821]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 4551 2588  143 2559]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  2788  778 3844 4269 1582 2154]
 [   0    0    0    0    0    0    0    0    0    0 2787 2324 3796 1020
   425   32 2324 3889 3669 3939]]


In [38]:
len(embedded_docs)

18285

## Creating Model

In [39]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_len))
model.add(LSTM(100)) # 1 LSTM layer having 100 neurons
model.add(Dense(1,activation='sigmoid')) # coz of classification prob
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [41]:
print(X_final.shape, Y_final.shape)

(18285, 20) (18285,)


In [42]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_final, Y_final, 
                                                test_size = 0.3, random_state=42)

## Model Training

In [43]:
model.fit(Xtrain,Ytrain,validation_data = (Xtest,Ytest), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 11s 39ms/step - loss: 0.3224 - accuracy: 0.8501 - val_loss: 0.1920 - val_accuracy: 0.9183
Epoch 2/10
200/200 [==============================] - 6s 32ms/step - loss: 0.1345 - accuracy: 0.9466 - val_loss: 0.1930 - val_accuracy: 0.9213
Epoch 3/10
200/200 [==============================] - 8s 39ms/step - loss: 0.0906 - accuracy: 0.9655 - val_loss: 0.2200 - val_accuracy: 0.9207
Epoch 4/10
200/200 [==============================] - 8s 39ms/step - loss: 0.0553 - accuracy: 0.9804 - val_loss: 0.2815 - val_accuracy: 0.9127
Epoch 5/10
200/200 [==============================] - 8s 38ms/step - loss: 0.0363 - accuracy: 0.9891 - val_loss: 0.2979 - val_accuracy: 0.9129
Epoch 6/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0209 - accuracy: 0.9934 - val_loss: 0.3516 - val_accuracy: 0.9112
Epoch 7/10
200/200 [==============================] - 6s 30ms/step - loss: 0.0105 - accuracy: 0.9972 - val_loss: 0.3826 - val_accuracy: 0.910

In [44]:
# Since the accuracy is 0.9996 almost 1.0 it may lead to overfitting prob

## Performance Metrics and Accuracy

In [46]:
import numpy as np
y_pred = model.predict(Xtest)
y_pred = np.round(y_pred).astype(int)

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Ytest,y_pred)
# False + and False - are very less compare to True + and True -

array([[2816,  291],
       [ 164, 2215]], dtype=int64)

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,y_pred)

0.9170616113744076

## Adding Dropout (Optional)

In [49]:
from tensorflow.keras.layers import Dropout
## creating Model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_len))
model.add(LSTM(150)) # 1 LSTM layer having 100 neurons
model.add(Dropout(0.6))
model.add(Dense(1,activation='sigmoid')) # coz of classification prob
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 150)               114600    
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 151       
                                                                 
Total params: 314,751
Trainable params: 314,751
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
model.fit(Xtrain,Ytrain,validation_data = (Xtest,Ytest), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 13s 49ms/step - loss: 0.3226 - accuracy: 0.8516 - val_loss: 0.1913 - val_accuracy: 0.9183
Epoch 2/10
200/200 [==============================] - 9s 47ms/step - loss: 0.1409 - accuracy: 0.9449 - val_loss: 0.2125 - val_accuracy: 0.9076
Epoch 3/10
200/200 [==============================] - 9s 44ms/step - loss: 0.1027 - accuracy: 0.9619 - val_loss: 0.2293 - val_accuracy: 0.9180
Epoch 4/10
200/200 [==============================] - 9s 44ms/step - loss: 0.0783 - accuracy: 0.9725 - val_loss: 0.2344 - val_accuracy: 0.9125
Epoch 5/10
200/200 [==============================] - 9s 43ms/step - loss: 0.0541 - accuracy: 0.9805 - val_loss: 0.3100 - val_accuracy: 0.9140
Epoch 6/10
200/200 [==============================] - 8s 40ms/step - loss: 0.0380 - accuracy: 0.9878 - val_loss: 0.3496 - val_accuracy: 0.9129
Epoch 7/10
200/200 [==============================] - 8s 40ms/step - loss: 0.0276 - accuracy: 0.9910 - val_loss: 0.3787 - val_accuracy: 0.909